In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date
from datetime import datetime
import re

In [2]:
el_diario = pd.read_csv("data/el_diario.csv")
el_diario.drop("Unnamed: 0",axis=1,inplace=True)
el_diario.to_csv("data/bups/el_diario_links_bup.csv")
old_links = el_diario.url.to_list()

In [3]:
new_links = pd.DataFrame(columns=["seccion","titulo","url","cmp","idx","dominio","crawl_day"])

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

url = "https://www.eldiario.es/"
driver.get(url)

page_source = driver.page_source

soup = BeautifulSoup(page_source, "html.parser") # or lxml

tags = soup.find_all("h2", {"class": "ni-title"})
errors=[]

for i in tags:
    l=[]
    
    cmp = i.a["cmp-ltrk"]
    idx = i.a["cmp-ltrk-idx"]
    href = i.a["href"]
    if href.startswith("http"):
        link = href
        dominio = href.split("/")[2]
    else:
        link = "https://www.eldiario.es{0}".format(i.a["href"])
        dominio = "www.eldiario.es"
    titulo = i.text
    seccion = i.a["href"].split("/")[1]
    
    l.append([seccion,titulo,link,cmp,idx,dominio,date.today()])
    df_ = pd.DataFrame(l,columns=["seccion","titulo","url","cmp","idx","dominio","crawl_day"])
    new_links = pd.concat([new_links,df_],axis=0)

new_links = new_links[new_links["dominio"] == "www.eldiario.es"]
new_links.reset_index(inplace=True,drop=True)

for col,row in new_links.iterrows():
    if row["url"] in old_links:
        new_links.drop(col,inplace=True)

el_diario = pd.concat([el_diario,new_links],axis=0)
el_diario.reset_index(inplace=True,drop=True)

print("{0} new links will be parsed".format(len(new_links)))

2 new links will be parsed


In [26]:
df_full = pd.DataFrame(columns=["url","seccion","cmp","idx","date","title","sub","sub_2","authors","text","crawl_day"])

for col,row in new_links.iterrows():
    l=[]
    url = row["url"]
    response = requests.get(url)
    if response.status_code != 404:
        soup = BeautifulSoup(response.text, "html.parser")

        if not "ultima-hora" in url.split("/")[4]:
            head = soup.find("head")

            d = soup.find("time",{"class": "date"})
            try:
                date = datetime.strptime(d["datetime"], '%Y-%m-%dT%H:%M:%S+02:00').date()
            except ValueError:
                date = datetime.strptime(d["datetime"], '%Y-%m-%dT%H:%M:%S+01:00').date()
            except TypeError:
                date = datetime.strptime(head.find_all("meta",property="article:published_time")[0]["content"],'%Y-%m-%dT%H:%M:%S+02:00').date()

            title = soup.find("h1",{"class": "title"})
            title = title.text.replace("\n"," ")
            title = re.sub(' +', ' ', title)
            
            try:
                ul = soup.find("ul",{"class": "footer"})
                ul = ul.text.replace("\n"," ")
                ul = re.sub(' +', ' ', ul)
                sub = ul.split("—  ")[0]
                try:
                    sub_2 = ul.split("—")[1]
                except IndexError:
                    sub_2 = "no sub_2"
            except AttributeError:
                pass
            
            try:
                author = soup.find("p",{"class": "authors"})
                author = author.text.replace("\n"," ")
            except AttributeError:
                try:
                    author = soup.find("a",{"class": "featured-name"})
                    author = author.text.replace("\n"," ")
                except AttributeError:
                    pass
            t = soup.find_all("p",{"class": "article-text"})
            text = ""

            for i in t:
                text = text + i.text
            text = text.replace("\n"," ")
            text = re.sub(' +', ' ', text)
            
            l.append([row["url"],row["seccion"],row["cmp"],row["idx"], date, title, sub, sub_2, author, text,date.today()])
            df_ = pd.DataFrame(l,columns=["url","seccion","cmp","idx","date","title","sub","sub_2","authors","text","crawl_day"])
            df_full = pd.concat([df_full,df_], axis=0)
    else:
        pass    

In [27]:
df_el_diario = pd.read_csv("data/el_diario_full.csv")
df_el_diario.drop("Unnamed: 0",axis=1,inplace=True)
df_el_diario.reset_index(inplace=True,drop=True)
df_el_diario.to_csv("data/bups/el_diario_full_bup.csv")

df_el_diario = pd.concat([df_el_diario,df_full],axis=0)
df_el_diario.reset_index(inplace=True,drop=True)
df_el_diario.to_csv("data/el_diario_full.csv")

# ACTUALIZANDO OLD LINKS
el_diario.to_csv("data/el_diario.csv")

In [6]:
df_el_diario = pd.read_csv("/Users/adriansanchezdelasierra/projects/news_parser/new_crawler/csvs/el_diario_full.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/csvs/el_diario_full.csv'

In [4]:
df_el_diario

,Unnamed: 0,url,seccion,cmp,idx,date,title,sub,sub_2,authors,text,crawl_day
0,0,https://www.eldiario.es/sociedad/educacion-no-...,sociedad,Apertura,0,2022-10-01,La Educación no consigue recuperarse de los r...,Desde las bajadas salariales al profesorado h...,Las familias invierten más cada año en Educac...,Daniel Sánchez Caballero,La lista es larga y sus consecuencias lo fuer...,2022-10-03
1,1,https://www.eldiario.es/economia/rebajas-fisca...,economia,Apertura,2,2022-10-01,Las rebajas fiscales evidencian la ideología ...,El Gobierno central responde al torrente de a...,Cómo te afecta la reforma fiscal del Gobierno...,Diego Larrouy,Terminan dos semanas intensas en el ámbito fi...,2022-10-03
2,2,https://www.eldiario.es/politica/feijoo-cobija...,politica,Apertura,5,2022-10-01,Feijóo se cobija en el bipartidismo y la élit...,"El líder del PP, que no desprende entusiasmo ...",González y Rajoy convergen en la crítica a la...,Esther Palomera,Sus más firmes entusiastas dicen que es un oa...,2022-10-03
3,3,https://www.eldiario.es/internacional/lula-enf...,internacional,Apertura,8,2022-10-01,Lula se enfrenta a Bolsonaro en un duelo hist...,El rechazo por Jair Bolsonaro ha acercado al ...,"Elecciones en Brasil: qué se vota, cómo se el...",Ayelén Oliva / Enviada especial,"Fernando Henrique Cardoso, líder del Partido ...",2022-10-03
4,4,https://www.eldiario.es/politica/ansia-reducir...,politica,Apertura,11,2022-10-01,El ansia por reducir impuestos consigue que L...,"En España y Reino Unido, la derecha insiste e...",no sub_2,Iñigo Sáenz de Ugarte,Isabel Díaz Ayuso recibió encantada la notici...,2022-10-03
...,...,...,...,...,...,...,...,...,...,...,...,...
931,931,https://www.eldiario.es/madrid/gerente-atencio...,madrid,Poti,8,2022-10-17,La gerente de Atención Primaria de Ayuso pres...,Sonia Martínez Machuca ha presentado su renun...,Huelga indefinida de sanitarios por los recor...,David Noriega,La gerente asistencial de Atención Primaria d...,2022-10-17
932,932,https://www.eldiario.es/comunitat-valenciana/j...,comunitat-valenciana,Poti,14,2022-10-17,Un joven denuncia una agresión homófoba grupa...,El joven fue agredido por seis personas hasta...,no sub_2,Laura Martínez,"“Maricón”, “bujarra”, “engendro”, “te vamos a...",2022-10-17
933,933,https://www.eldiario.es/carnecruda/programas/t...,carnecruda,home-sin-identificar,6,2022-10-14,Que trabajen los robots,Imaginamos Mundos Posibles con Esther Sánchez...,no sub_2,NaN,"Escúchalo en iVoox, Spotify o tu plataforma d...",2022-10-17
934,934,https://www.eldiario.es/madrid/gerente-atencio...,madrid,edicion-local,0,2022-10-17,La gerente de Atención Primaria de Ayuso pres...,Sonia Martínez Machuca ha presentado su renun...,Huelga indefinida de sanitarios por los recor...,David Noriega,La gerente asistencial de Atención Primaria d...,2022-10-17
